In [ ]:
import glob
import os

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import bilby
import corner
import h5py
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import numpy.lib.recfunctions as rfn
from scipy import stats
import seaborn as sns
from pesummary.core.plots.publication import reverse_triangle_plot


from thesis_utils.gw import get_cbc_parameter_labels
from thesis_utils.plotting import (
    set_plotting,
    save_figure,
    get_default_corner_kwargs,
    get_default_figsize,
)

set_plotting()

In [ ]:
gwtc2p1_file = "/home/michael.williams/git_repos/nessai-gwtc-1/gwtc-2.1_sample_release/IGWN-GWTC2p1-v2-GW190521_030229_PEDataRelease_mixed_nocosmo.h5"
with h5py.File(gwtc2p1_file, "r") as f:
    print(f["C01:IMRPhenomXPHM"].keys())
    gwtc2p1_XPHM_posterior = f["C01:IMRPhenomXPHM/posterior_samples"][()]

In [ ]:
with h5py.File("GW190521/GW190521_posterior_samples.h5", "r") as f:
    print(f.keys())
    discovery_samples = f["SEOBNRv4PHM/posterior_samples"][()]

In [ ]:
nessai_result = bilby.core.result.read_in_result(
    "../analysis/O3/outdir_GW190521_IMRPhenomXPHM/final_result/GW190521_IMRPhenomXPHM_data0_1242442967-459473_analysis_H1L1V1_merge_result.hdf5"
)
nessai_result_nlive = bilby.core.result.read_in_result(
    "../analysis/O3/outdir_GW190521_IMRPhenomXPHM_v1/final_result/GW190521_IMRPhenomXPHM_data0_1242442967-459473_analysis_H1L1V1_merge_result.hdf5"
)

In [ ]:
nessai_result.posterior.keys()
if "chi_eff" not in nessai_result.posterior.keys():
    nessai_result.posterior = bilby.gw.conversion.generate_all_bbh_parameters(
        nessai_result.posterior
    )

In [ ]:
nessai_result_nlive.posterior.keys()
if "chi_eff" not in nessai_result_nlive.posterior.keys():
    nessai_result_nlive.posterior = bilby.gw.conversion.generate_all_bbh_parameters(
        nessai_result_nlive.posterior
    )

In [ ]:
parameters = ["chirp_mass", "mass_ratio", "chi_eff", "theta_jn"]

corner_kwargs = get_default_corner_kwargs()
corner_kwargs["plot_density"] = True
corner_kwargs["plot_datapoints"] = False
corner_kwargs["no_fill_contours"] = True
corner_kwargs["show_titles"] = False
corner_kwargs.pop("fill_contours")
corner_kwargs["levels"] = (1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.0))
corner_kwargs["quantiles"] = []
corner_kwargs["smooth"] = 0.9
corner_kwargs["bins"] = 32
fig = None

corner_kwargs["color"] = "k"
corner_kwargs["hist_kwargs"]["color"] = "k"
corner_kwargs["hist_kwargs"]["ls"] = "--"
fig = corner.corner(
    np.array([gwtc2p1_XPHM_posterior[p] for p in parameters]).T,
    fig=fig,
    contour_kwargs=dict(linestyles=["--"]),
    **corner_kwargs,
)

corner_kwargs["hist_kwargs"]["ls"] = "-"
corner_kwargs["color"] = "C0"
corner_kwargs["hist_kwargs"]["color"] = "C0"
fig = corner.corner(
    np.array([nessai_result.posterior[p] for p in parameters]).T,
    fig=fig,
    labels=get_cbc_parameter_labels(parameters, units=True),
    labelpad=-0.1,
    **corner_kwargs,
)

corner_kwargs["hist_kwargs"]["ls"] = "-"
corner_kwargs["color"] = "C1"
corner_kwargs["hist_kwargs"]["color"] = "C1"
fig = corner.corner(
    np.array([nessai_result_nlive.posterior[p] for p in parameters]).T,
    fig=fig,
    labels=get_cbc_parameter_labels(parameters, units=True),
    labelpad=-0.1,
    **corner_kwargs,
)

legend_elements = [
    Line2D([0], [0], color="k", ls="--", label=r"GWTC-2.1"),
    Line2D(
        [0], [0], color="C0", ls="-", label=r"\texttt{nessai} - $N_\textrm{live}=1000$"
    ),
    Line2D(
        [0], [0], color="C1", ls="-", label=r"\texttt{nessai} - $N_\textrm{live}=2000$"
    ),
]

fig.legend(
    handles=legend_elements,
    bbox_to_anchor=(1.0, 0.9),
    loc="upper right",
    fontsize=18,
)
plt.show()
save_figure(fig, "GW190521_posterior", "figures/GW190521/")

In [ ]:
nessai_result_paths = {
    "default": "/home/michael.williams/git_repos/nessai-gwtc-1/analysis/O3/outdir_GW190521_IMRPhenomXPHM/result/GW190521_IMRPhenomXPHM_data0_1242442967-459473_analysis_H1L1V1*_nessai/result.hdf5",
    "nlive": "/home/michael.williams/git_repos/nessai-gwtc-1/analysis/O3/outdir_GW190521_IMRPhenomXPHM_v1/result/GW190521_IMRPhenomXPHM_data0_1242442967-459473_analysis_H1L1V1*_nessai/result.hdf5",
}
indices = {}
for key, path in nessai_result_paths.items():
    indices[key] = []
    nessai_result_files = glob.glob(path)
    for rf in nessai_result_files:
        with h5py.File(rf, "r") as f:
            indices[key].append(f["insertion_indices"][()])

In [ ]:
indices

In [ ]:
figsize = get_default_figsize()
figsize[1] *= 0.7
fig, axs = plt.subplots(1, 2, figsize=figsize, sharey=True)

colours = {
    "default": sns.color_palette("Blues", n_colors=3)[1:],
    "nlive": sns.color_palette("Oranges", n_colors=3)[1:],
}

for (key, indices_data), (j, nlive) in zip(indices.items(), enumerate([1000, 2000])):
    # indices_data = indices["nlive"]

    # plt.axhline(0, color="grey")
    x = np.arange(0, nlive, 1)
    analytic = x / x[-1]
    n = np.mean([len(idx) for idx in indices_data])
    c = colours.get(key)
    for i, idx in enumerate(indices_data):
        _, counts = np.unique(idx, return_counts=True)
        estimated = np.cumsum(counts) / len(idx)
        axs[j].plot(analytic - estimated, ls="-", c=c[i])

    for ci in [0.5, 0.95, 0.997]:
        bound = (1 - ci) / 2
        bound_values = stats.binom.ppf(1 - bound, n, analytic) / n
        lower = bound_values - analytic
        upper = analytic - bound_values
        upper[0] = 0
        upper[-1] = 0
        lower[0] = 0
        lower[-1] = 0

        axs[j].fill_between(x, lower, upper, color="grey", alpha=0.2)

    axs[j].set_xlim(0, nlive)
    axs[j].set_xlabel("Insertion index")
    axs[j].set_title(r"$N_\textrm{live}=" + str(nlive) + r"$", loc="left")
axs[0].set_ylabel(r"Analytic CMF - Empirical CMF")
plt.tight_layout()
plt.show()

save_figure(fig, f"GW190521_insertion_indices", "figures/GW190521/")
# fig.savefig("figures/GW190521/insertion_indices.png", dpi=200, bbox_inches="tight")